## INTALANDO SPARK

In [1]:
%pip install pyspark==3.3.1


[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


### IMPORTS 

importando informações para criar uam spark session

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as f

from scripts.load_silver_functions import *

### CRIANDO UMA SESSION

In [3]:
spark = SparkSession\
    .builder\
    .appName("twitter_transformation")\
    .getOrCreate()

24/09/02 19:27:05 WARN Utils: Your hostname, kaiwangairflow resolves to a loopback address: 127.0.1.1; using 192.168.2.128 instead (on interface ens33)
24/09/02 19:27:05 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


24/09/02 19:27:06 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


#### LENDO ARQUIVOS JSON

In [4]:
df = spark.read.json("../../datalake/twitter_dataengineer")

In [5]:
df.show()

+--------------------+--------------------+------------------+------------+
|                data|            includes|              meta|extract_date|
+--------------------+--------------------+------------------+------------+
|[{89, 3, 2024-08-...|{[{2024-08-23T21:...|{1234567890abcdef}|  2024-08-23|
|[{30, 18, 2024-08...|{[{2024-08-23T04:...|{1234567890abcdef}|  2024-08-23|
|[{58, 13, 2024-08...|{[{2024-08-23T17:...|              null|  2024-08-23|
|[{46, 77, 2024-08...|{[{2024-08-23T03:...|{1234567890abcdef}|  2024-08-23|
|[{32, 73, 2024-08...|{[{2024-08-23T23:...|{1234567890abcdef}|  2024-08-23|
|[{42, 100, 2024-0...|{[{2024-08-23T13:...|              null|  2024-08-23|
|[{38, 69, 2024-08...|{[{2024-08-23T16:...|              null|  2024-08-23|
|[{91, 12, 2024-08...|{[{2024-08-23T15:...|              null|  2024-08-23|
|[{58, 51, 2024-08...|{[{2024-08-23T19:...|              null|  2024-08-23|
|[{48, 22, 2024-08...|{[{2024-08-23T01:...|{1234567890abcdef}|  2024-08-23|
|[{33, 60, 2

In [6]:
df.printSchema()

root
 |-- data: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- author_id: string (nullable = true)
 |    |    |-- conversation_id: string (nullable = true)
 |    |    |-- created_at: string (nullable = true)
 |    |    |-- edit_history_tweet_ids: array (nullable = true)
 |    |    |    |-- element: long (containsNull = true)
 |    |    |-- id: string (nullable = true)
 |    |    |-- in_reply_to_user_id: string (nullable = true)
 |    |    |-- lang: string (nullable = true)
 |    |    |-- public_metrics: struct (nullable = true)
 |    |    |    |-- like_count: long (nullable = true)
 |    |    |    |-- quote_count: long (nullable = true)
 |    |    |    |-- reply_count: long (nullable = true)
 |    |    |    |-- retweet_count: long (nullable = true)
 |    |    |-- text: string (nullable = true)
 |-- includes: struct (nullable = true)
 |    |-- users: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- c

In [7]:
df.select(f.explode("data")).printSchema()

root
 |-- col: struct (nullable = true)
 |    |-- author_id: string (nullable = true)
 |    |-- conversation_id: string (nullable = true)
 |    |-- created_at: string (nullable = true)
 |    |-- edit_history_tweet_ids: array (nullable = true)
 |    |    |-- element: long (containsNull = true)
 |    |-- id: string (nullable = true)
 |    |-- in_reply_to_user_id: string (nullable = true)
 |    |-- lang: string (nullable = true)
 |    |-- public_metrics: struct (nullable = true)
 |    |    |-- like_count: long (nullable = true)
 |    |    |-- quote_count: long (nullable = true)
 |    |    |-- reply_count: long (nullable = true)
 |    |    |-- retweet_count: long (nullable = true)
 |    |-- text: string (nullable = true)



In [8]:
df.select(f.explode("data")).show()

+--------------------+
|                 col|
+--------------------+
|{89, 3, 2024-08-2...|
|{97, 83, 2024-08-...|
|{54, 18, 2024-08-...|
|{4, 79, 2024-08-2...|
|{74, 15, 2024-08-...|
|{1, 73, 2024-08-2...|
|{80, 21, 2024-08-...|
|{19, 3, 2024-08-2...|
|{5, 21, 2024-08-2...|
|{70, 51, 2024-08-...|
|{30, 18, 2024-08-...|
|{15, 40, 2024-08-...|
|{96, 60, 2024-08-...|
|{85, 42, 2024-08-...|
|{18, 10, 2024-08-...|
|{14, 98, 2024-08-...|
|{99, 58, 2024-08-...|
|{76, 65, 2024-08-...|
|{18, 84, 2024-08-...|
|{6, 3, 2024-08-23...|
+--------------------+
only showing top 20 rows



In [9]:
df.select(f.explode("data").alias("tweets"))\
.select("tweets.author_id", "tweets.conversation_id",
        "tweets.created_at", "tweets.id",
        "tweets.public_metrics.*", "tweets.text").printSchema()

root
 |-- author_id: string (nullable = true)
 |-- conversation_id: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- id: string (nullable = true)
 |-- like_count: long (nullable = true)
 |-- quote_count: long (nullable = true)
 |-- reply_count: long (nullable = true)
 |-- retweet_count: long (nullable = true)
 |-- text: string (nullable = true)



In [10]:
tweet_df = get_tweeters(df)

In [11]:
tweet_df.show()

+---------+---------------+--------------------+---+----------+-----------+-----------+-------------+--------------------+
|author_id|conversation_id|          created_at| id|like_count|quote_count|reply_count|retweet_count|                text|
+---------+---------------+--------------------+---+----------+-----------+-----------+-------------+--------------------+
|       89|              3|2024-08-23T04:43:...| 78|        52|         36|         59|           95|Outro tweet fictí...|
|       97|             83|2024-08-23T07:20:...|  0|        86|          9|         73|           88|Tweet fictício ge...|
|       54|             18|2024-08-23T04:43:...|100|        37|         23|         64|            3|Um terceiro tweet...|
|        4|             79|2024-08-23T06:29:...| 30|        49|         88|         88|           27|Tweet fictício cr...|
|       74|             15|2024-08-23T20:11:...| 88|        12|         90|         48|           30|Um terceiro tweet...|
|        1|     

In [12]:
df.select(f.explode("includes.users")).printSchema()




root
 |-- col: struct (nullable = true)
 |    |-- created_at: string (nullable = true)
 |    |-- id: string (nullable = true)
 |    |-- name: string (nullable = true)
 |    |-- username: string (nullable = true)



In [13]:
users_df = get_users(df)

In [14]:
users_df.show()

+--------------------+---+-------+--------+
|          created_at| id|   name|username|
+--------------------+---+-------+--------+
|2024-08-23T21:36:...| 48| User 1|   user1|
|2024-08-23T14:48:...| 67| User 2|   user2|
|2024-08-23T09:15:...| 89| User 3|   user3|
|2024-08-23T22:34:...| 99| User 4|   user4|
|2024-08-23T04:47:...| 47| User 5|   user5|
|2024-08-23T20:19:...| 15| User 6|   user6|
|2024-08-23T21:22:...|  4| User 7|   user7|
|2024-08-23T04:03:...| 48| User 8|   user8|
|2024-08-23T09:34:...| 90| User 9|   user9|
|2024-08-23T11:49:...| 35|User 10|  user10|
|2024-08-23T04:03:...| 92| User 1|   user1|
|2024-08-23T08:20:...| 84| User 2|   user2|
|2024-08-23T22:38:...| 57| User 3|   user3|
|2024-08-23T08:55:...| 61| User 4|   user4|
|2024-08-23T18:28:...| 79| User 5|   user5|
|2024-08-23T18:51:...| 48| User 6|   user6|
|2024-08-23T14:51:...| 40| User 7|   user7|
|2024-08-23T18:08:...| 32| User 8|   user8|
|2024-08-23T16:22:...| 84| User 9|   user9|
|2024-08-23T16:51:...| 45|User 1

In [15]:
tweet_df.coalesce(1).write.mode("overwrite").json('output/tweet')
users_df.coalesce(1).write.mode("overwrite").json('output/user')